In [2]:
! git clone https://github.com/jortizta/atmos-neuralnet

Cloning into 'atmos-neuralnet'...
remote: Enumerating objects: 2881, done.
remote: Counting objects: 100% (2881/2881), done.
remote: Compressing objects: 100% (2866/2866), done.
remote: Total 2881 (delta 21), reused 2862 (delta 12), pack-reused 0
Receiving objects: 100% (2881/2881), 23.75 MiB | 10.24 MiB/s, done.
Resolving deltas: 100% (21/21), done.


In [3]:
cd atmos-neuralnet/src

/content/atmos-neuralnet/src


In [0]:
from net import TurbNetG, weights_init
import os, sys, random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
import torch.optim as optim
import matplotlib.pyplot as plt
import dataset
import utils

In [0]:
# number of training iterations
iterations = 10000
# batch size
batch_size = 10
# learning rate, generator
lrG = 0.0006
# decay learning rate?
decayLr = True
# channel exponent to control network size
expo = 5
# data set config
prop=None # by default, use all from "../data/train"
#prop=[1000,0.75,0,0.25] # mix data from multiple directories
# save txt files with per epoch loss?
saveL1 = False


In [0]:
autoIter   = False
dropout    = 0.
doLoad     = ""


In [7]:
seed = random.randint(0, 2**32 - 1)
print("Random seed: {}".format(seed))
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

Random seed: 4071311880


In [8]:
data = dataset.TurbDataset(shuffle=1)
trainLoader = DataLoader(data, batch_size=batch_size, shuffle=True, drop_last=True)
print("Training batches: {}".format(len(trainLoader)))
dataValidation = dataset.ValiDataset(data)
valiLoader = DataLoader(dataValidation, batch_size=batch_size, shuffle=False, drop_last=True)
print("Validation batches: {}".format(len(valiLoader)))

Training batches: 102
Validation batches: 25


/content/atmos-neuralnet/src/dataset.py:69: RuntimeWarning: invalid value encountered in true_divide
  rawDataNorm= rawDataF/sDev[np.newaxis,:,:]


In [9]:
# setup training
epochs = int(iterations/len(trainLoader) + 0.5)
netG = TurbNetG(channelExponent=expo, dropout=dropout)
print(netG) # print full net
model_parameters = filter(lambda p: p.requires_grad, netG.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print("Initialized TurbNet with {} trainable params ".format(params))

netG.apply(weights_init)
if len(doLoad)>0:
    netG.load_state_dict(torch.load(doLoad))
    print("Loaded model "+doLoad)
netG.cuda()

criterionL1 = nn.L1Loss()
criterionL1.cuda()

optimizerG = optim.Adam(netG.parameters(), lr=lrG, betas=(0.5, 0.999), weight_decay=0.0)

targets = Variable(torch.FloatTensor(batch_size, 2, 128, 1))
inputs  = Variable(torch.FloatTensor(batch_size, 5, 128, 1))
targets = targets.cuda()
inputs  = inputs.cuda()


TurbNetG(
  (layer1): Sequential(
    (layer1_conv): Conv2d(5, 32, kernel_size=[4, 1], stride=(2, 2), padding=[1, 0])
  )
  (layer2): Sequential(
    (layer2_leakyrelu): LeakyReLU(negative_slope=0.2, inplace)
    (layer2_conv): Conv2d(32, 64, kernel_size=[4, 1], stride=(2, 2), padding=[1, 0])
    (layer2_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (layer2b): Sequential(
    (layer2b_leakyrelu): LeakyReLU(negative_slope=0.2, inplace)
    (layer2b_conv): Conv2d(64, 64, kernel_size=[4, 1], stride=(2, 2), padding=[1, 0])
    (layer2b_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (layer3): Sequential(
    (layer3_leakyrelu): LeakyReLU(negative_slope=0.2, inplace)
    (layer3_conv): Conv2d(64, 128, kernel_size=[4, 1], stride=(2, 2), padding=[1, 0])
    (layer3_bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (layer4): Sequential(
    (layer4_leakyrelu): Leaky

In [10]:
for epoch in range(epochs):
    print("Starting epoch {} / {}".format((epoch+1),epochs))

    netG.train()
    L1_accum = 0.0
    samples_accum = 0
    for i, traindata in enumerate(trainLoader, 0):

        inputs_cpu, targets_cpu = traindata
        current_batch_size = targets_cpu.size(0)

        targets_cpu, inputs_cpu = targets_cpu.float().cuda(), inputs_cpu.float().cuda()
        inputs.data.resize_as_(inputs_cpu).copy_(inputs_cpu)
        targets.data.resize_as_(targets_cpu).copy_(targets_cpu)

        # compute LR decay
        if decayLr:
            currLr = utils.computeLR(epoch, epochs, lrG*0.1, lrG)
            if currLr < lrG:
                for g in optimizerG.param_groups:
                    g['lr'] = currLr

        netG.zero_grad()
        gen_out = netG(inputs)

        lossL1 = criterionL1(gen_out, targets)
        lossL1.backward()

        optimizerG.step()

        lossL1viz = lossL1.item()
        L1_accum += lossL1viz
        samples_accum += current_batch_size



        if i==len(trainLoader)-1:
            logline = "Epoch: {}, batch-idx: {}, L1: {}\n".format(epoch, i, lossL1viz)
            #print(logline)


    # validation
    netG.eval()
    L1val_accum = 0.0
    for i, validata in enumerate(valiLoader, 0):
        inputs_cpu, targets_cpu = validata
        current_batch_size = targets_cpu.size(0)

        targets_cpu, inputs_cpu = targets_cpu.float().cuda(), inputs_cpu.float().cuda()
        inputs.data.resize_as_(inputs_cpu).copy_(inputs_cpu)
        targets.data.resize_as_(targets_cpu).copy_(targets_cpu)

        outputs = netG(inputs)
        outputs_cpu = outputs.data.cpu().numpy()

        lossL1 = criterionL1(outputs, targets)
        L1val_accum += lossL1.item()


        if i==0:
            input_ndarray = inputs_cpu.cpu().numpy()[0]
            v_norm = ( np.max(np.abs(input_ndarray[0,:,:]))**2 + np.max(np.abs(input_ndarray[1,:,:]))**2 )**0.5
            #print(outputs[0]-targets[0])
            #print(targets[0])

            #outputs_denormalized = data.denormalize(outputs_cpu[0], v_norm)
            #targets_denormalized = data.denormalize(targets_cpu.cpu().numpy()[0], v_norm)
            #utils.makeDirs(["results_train"])
            #utils.imageOut("results_train/epoch{}_{}".format(epoch, i), outputs_denormalized, targets_denormalized, saveTargets=True)

    # data for graph plotting
    L1_accum    /= len(trainLoader)
    L1val_accum /= len(valiLoader)
    if saveL1:
        if epoch==0:
            utils.resetLog(prefix + "L1.txt"   )
            utils.resetLog(prefix + "L1val.txt")
        utils.log(prefix + "L1.txt"   , "{} ".format(L1_accum), False)
        utils.log(prefix + "L1val.txt", "{} ".format(L1val_accum), False)

Starting epoch 1 / 98
Starting epoch 2 / 98
Starting epoch 3 / 98
Starting epoch 4 / 98
Starting epoch 5 / 98
Starting epoch 6 / 98
Starting epoch 7 / 98
Starting epoch 8 / 98
Starting epoch 9 / 98
Starting epoch 10 / 98
Starting epoch 11 / 98
Starting epoch 12 / 98
Starting epoch 13 / 98
Starting epoch 14 / 98
Starting epoch 15 / 98
Starting epoch 16 / 98
Starting epoch 17 / 98
Starting epoch 18 / 98
Starting epoch 19 / 98
Starting epoch 20 / 98
Starting epoch 21 / 98
Starting epoch 22 / 98
Starting epoch 23 / 98
Starting epoch 24 / 98
Starting epoch 25 / 98
Starting epoch 26 / 98
Starting epoch 27 / 98
Starting epoch 28 / 98
Starting epoch 29 / 98
Starting epoch 30 / 98
Starting epoch 31 / 98
Starting epoch 32 / 98
Starting epoch 33 / 98
Starting epoch 34 / 98
Starting epoch 35 / 98
Starting epoch 36 / 98
Starting epoch 37 / 98
Starting epoch 38 / 98
Starting epoch 39 / 98
Starting epoch 40 / 98
Starting epoch 41 / 98
Starting epoch 42 / 98
Starting epoch 43 / 98
Starting epoch 44 / 

In [11]:
ls

dataset.py  progressbar.py  training_v0.ipynb  utils.py
net.py      __pycache__/    training_v0.py
